# Source: Sports Reference - NFL Data

In [1]:
#hide
import json
import core_constants as cc
import functions as fx
import pandas as pd
import sqlite3 as sql

## Set Notebook Settings

In [2]:
years = cc.get_defYears()
headers= cc.get_header()
schoolsList = cc.get_schoolsList()

## Get, Process the NFL draft data from SR
#### Source: https://www.pro-football-reference.com/years/2017/draft.htm
> This page contains metadata of each draft pick, both in terms of draft position but also current pro stats.

In [3]:
with open("..//scrapedData//nfldraft.json", "w", encoding="utf-8") as write_file:
                write_file.write(json.dumps(fx.handle_nflData(years, headers, schoolsList)))

## Save to DB

In [4]:
fx.toDB_NFLDraft()

'DB Write is done'

In [7]:
#Output School List
listSchools = []
for i in schoolsList:
    listSchools.append(i['id'])

listSchools.sort()
for i in listSchools:
    print(i)

air-force
akron
alabama
alabama-birmingham
appalachian-state
arizona
arizona-state
arkansas
arkansas-state
army
auburn
ball-state
baylor
boise-state
boston-college
bowling-green
buffalo
byu
california
central-florida
central-michigan
charlotte
cincinnati
clemson
coastal-carolina
colorado
colorado-state
duke
east-carolina
eastern-michigan
florida
florida-atlantic
florida-international
florida-state
fresno-state
georgia
georgia-southern
georgia-state
georgia-tech
hawaii
houston
illinois
indiana
iowa
iowa-state
kansas
kansas-state
kent-state
kentucky
liberty
louisiana
louisiana-monroe
louisiana-tech
louisville
lsu
marshall
maryland
massachusetts
memphis
miami
michigan
michigan-state
middle-tennessee-state
minnesota
mississippi-state
missouri
navy
nebraska
nevada
new-mexico
new-mexico-state
north-carolina
north-carolina-state
north-texas
northern-illinois
northwestern
notre-dame
ohio
ohio-state
oklahoma
oklahoma-state
old-dominion
ole-miss
oregon
oregon-state
penn-state
pittsburgh
purdue
r